In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import json
import requests
import webbrowser
from urllib.parse import urlencode
import time 

In [3]:
load_dotenv("untitled.env")

True

In [5]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
access_token = "access_token"
refresh_token = "refresh_token"
playlist_id = os.getenv("playlist_id")

In [16]:
import requests
import time

def refresh_access_token(refresh_token, client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def get_playlist_track_ids(playlist_id, access_token, refresh_token, client_id, client_secret, limit=100):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "limit": limit
    }
    track_ids = []
    while url:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 429:  # Rate limit exceeded
            retry_after = int(response.headers.get('Retry-After', 1))
            print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            continue
        elif response.status_code == 401:  # Unauthorized, token might have expired
            print("Access token expired, refreshing...")
            token_data = refresh_access_token(refresh_token, client_id, client_secret)
            if token_data and 'access_token' in token_data:
                access_token = token_data['access_token']
                headers['Authorization'] = f"Bearer {access_token}"
                response = requests.get(url, headers=headers, params=params)
            else:
                print("Failed to refresh access token")
                break

        if response.status_code == 200:
            data = response.json()
            track_ids.extend(item['track']['id'] for item in data['items'] if item['track'])
            url = data['next']  # Get the next page URL
            params = None  # Only needed for the first request
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    return track_ids




track_ids = get_playlist_track_ids(playlist_id, access_token, refresh_token, client_id, client_secret)
print(f"Collected {len(track_ids)} track IDs")


Collected 945 track IDs


In [18]:
file_path = 'track_ids.txt'

# Write the list to the text file
with open(file_path, 'w') as file:
    for track_id in track_ids:
        file.write(track_id + '\n')

print(f'Track IDs have been saved to {file_path}')

Track IDs have been saved to track_ids.txt


In [7]:
file_path = 'track_ids.txt'

# Initialize an empty list to hold the track IDs
track_ids = []

# Read the file and populate the list
with open(file_path, 'r') as file:
    for line in file:
        # Strip any whitespace characters like \n at the end of each line
        track_id = line.strip()
        # Add the track ID to the list
        track_ids.append(track_id)
track_ids

['6Ep6BzIOB9tz3P4sWqiiAB',
 '0RchWnEHETINMAyrTrlBmo',
 '1r1fPuhj9H4VdXr7OK6FL5',
 '1NNAI51EuoRWw1ydX1zV7S',
 '0qwEthxOWMmIrPTjTWa2Cb',
 '6nYD1UPZzBfnA9aL5TrWTc',
 '46IZ0fSY2mpAiktS3KOqds',
 '3di5hcvxxciiqwMH1jarhY',
 '3qcpRuoY629US8hlDAQkNk',
 '2JzZzZUQj3Qff7wapcbKjc',
 '5edBgVtRD0fvWk140Sl21T',
 '2aFiaMXmWsM3Vj72F9ksBl',
 '3RiPr603aXAoi4GHyXx0uy',
 '7Izh7zLJGw1Nscd62Exxj4',
 '3BfUhr0qu1H1pQdoJdWgLn',
 '2L6hCTpUlR2p7Su3hiSB0s',
 '2krwNGlAHQuClcMw0WMUiz',
 '6Sr3iRWoMsyqVfNDJJQmzk',
 '6oHDvarQSp0mf5AD1SyNH0',
 '5yKfZwcuSoEkOemLJX6hEm',
 '0pQr5WsobmdZHnuObT0sc2',
 '14SZQ3k55sfzGLuv9WZ4Xe',
 '7s3VxmIizcDgDTRCA4kDzd',
 '6yxQYuItvkkjEDGvORwzeD',
 '273QnyCvJB65rScHJ1nPZb',
 '3bNv3VuUOKgrf5hu3YcuRo',
 '4zzzZ1UNfr75ASG1lUE9L1',
 '1mKXFLRA179hdOWQBwUk9e',
 '7BqBn9nzAq8spo5e7cZ0dJ',
 '4HlFJV71xXKIGcU3kRyttv',
 '2hIY4sjfMeKwXXh2LvhrMo',
 '07TAUihsoCBnLWWXuZYUyw',
 '3GRoxbBauU8Ui1zUJwn9aD',
 '0tgVpDi06FyKpA1z0VMD4v',
 '519iK3BrSWhVLhmBlDIP4x',
 '0rKtyWc8bvkriBthvHKY8d',
 '3U4isOIWM3VvDubwSI3y7a',
 

In [9]:
def get_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode(),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None
client_id1 = "099284fdc42d44d3bf175e431a142c9c"
client_secret1 = "0f3d14a3979f428c99da2eb71b26bfc8"
client_credentials_token = get_access_token(client_id1, client_secret1)

In [11]:
client_credentials_token

'BQA3JkURmuZF_1yDP7gYG9Ujtiuj-PU5uLve78EBTLcQHi7tcOiyKQrRBGsWVbh8tMxYMlDYTP7znbWrdt41FrfXpoIO46iEFw0aDvPDuFl_B6tl7Kw'

In [13]:
def get_track_metadata(track_id, access_token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 429:  
        retry_after = int(response.headers.get('Retry-After', 1))
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def get_track_audio_features(track_id, access_token):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 429:  
        retry_after = int(response.headers.get('Retry-After', 1))
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def save_to_file(data, filename='track_data1.json'):
    with open(filename, 'a') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

In [15]:
track_data_list = []

for track_id in track_ids:
    track_metadata = get_track_metadata(track_id, client_credentials_token)
    track_audio_features = get_track_audio_features(track_id, client_credentials_token)
    if track_metadata and track_audio_features:
        track_data = {
            'metadata': {
                'id': track_id,
                'album_type': track_metadata['album']['album_type'],
                'album_name': track_metadata['album']['name'],
                'artists': [{'name': artist['name'], 'followers': artist.get('followers', {}).get('total')} for artist in track_metadata['artists']],
                'track_name': track_metadata['name'],
                'popularity': track_metadata['popularity'],
                'release_date': track_metadata['album']['release_date'],
            },
            'audio_features': {
                'danceability': track_audio_features['danceability'],
                'energy': track_audio_features['energy'],
                'key': track_audio_features['key'],
                'loudness': track_audio_features['loudness'],
                'mode': track_audio_features['mode'],
                'speechiness': track_audio_features['speechiness'],
                'acousticness': track_audio_features['acousticness'],
                'instrumentalness': track_audio_features['instrumentalness'],
                'liveness': track_audio_features['liveness'],
                'valence': track_audio_features['valence'],
                'tempo': track_audio_features['tempo'],
                'duration_ms': track_audio_features['duration_ms'],
                'time_signature': track_audio_features['time_signature']
            }
        }
        track_data_list.append(track_data)


Error: 404, {
  "error" : {
    "status" : 404,
    "message" : "Non existing id: 'spotify:track:4KRGLFU3OGqg64tfxI9Eki'"
  }
}
Error: 404, {
  "error" : {
    "status" : 404,
    "message" : "analysis not found"
  }
}


In [17]:
len(track_data_list)

944

In [25]:
track_data_list

[{'metadata': {'id': '6Ep6BzIOB9tz3P4sWqiiAB',
   'album_type': 'album',
   'album_name': 'Night Visions (Deluxe)',
   'artists': [{'name': 'Imagine Dragons', 'followers': None}],
   'track_name': 'Radioactive',
   'popularity': 0,
   'release_date': '2012-09-04'},
  'audio_features': {'danceability': 0.473,
   'energy': 0.777,
   'key': 9,
   'loudness': -3.698,
   'mode': 1,
   'speechiness': 0.059,
   'acousticness': 0.119,
   'instrumentalness': 7.87e-05,
   'liveness': 0.671,
   'valence': 0.21,
   'tempo': 136.249,
   'duration_ms': 186813,
   'time_signature': 4}},
 {'metadata': {'id': '0RchWnEHETINMAyrTrlBmo',
   'album_type': 'album',
   'album_name': 'Night Visions (Deluxe)',
   'artists': [{'name': 'Imagine Dragons', 'followers': None}],
   'track_name': 'Bleeding Out',
   'popularity': 0,
   'release_date': '2012-09-04'},
  'audio_features': {'danceability': 0.318,
   'energy': 0.749,
   'key': 5,
   'loudness': -4.045,
   'mode': 0,
   'speechiness': 0.0702,
   'acousticne

In [27]:
save_to_file(track_data_list)
print("Track metadata and audio features have been saved to track_data.json")

Track metadata and audio features have been saved to track_data.json


In [63]:
import pyodbc
from datetime import datetime
# Function to create tables in SQL Server database
def create_tables(server, database, driver):
    conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
    cursor = conn.cursor()

    # Create tracks table with surrogate key 'id' and 'track_id'
    cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='tracks')
        BEGIN
            CREATE TABLE tracks (
                id INT PRIMARY KEY IDENTITY(1,1),
                track_id VARCHAR(255) UNIQUE,
                album_type VARCHAR(1024),
                album_name VARCHAR(1024),
                track_name VARCHAR(1024),
                popularity INT,
                release_date VARCHAR(255)
            )
        END
    ''')

    # Create artists table 
    cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='artists')
        BEGIN
            CREATE TABLE artists (
                id INT PRIMARY KEY IDENTITY(1,1),
                track_id VARCHAR(255),
                artist_name VARCHAR(1024),
                artist_followers INT,
                FOREIGN KEY (track_id) REFERENCES tracks(track_id)
            )
        END
    ''')

    # Create audio_features table 
    cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='audio_features')
        BEGIN
            CREATE TABLE audio_features (
                id INT PRIMARY KEY IDENTITY(1,1),
                track_id VARCHAR(255) UNIQUE,
                danceability FLOAT,
                energy FLOAT,
                key_int INT,
                loudness FLOAT,
                mode_int INT,
                speechiness FLOAT,
                acousticness FLOAT,
                instrumentalness FLOAT,
                liveness FLOAT,
                valence FLOAT,
                tempo FLOAT,
                duration_ms INT,
                time_signature INT,
                FOREIGN KEY (track_id) REFERENCES tracks(track_id)
            )
        END
    ''')

    conn.commit()
    conn.close()


def insert_track_data(server, database, driver, track_data):
    conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
    cursor = conn.cursor()

    try:
        
        cursor.execute('''
            INSERT INTO tracks (track_id, album_type, album_name, track_name, popularity, release_date)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            track_data['metadata']['id'],
            track_data['metadata']['album_type'],
            track_data['metadata']['album_name'],
            track_data['metadata']['track_name'],
            track_data['metadata']['popularity'],
           track_data['metadata']['release_date']
        ))

        
        track_id = track_data['metadata']['id']

        
        artists = track_data['metadata'].get('artists', [])  
        for artist in artists:
            cursor.execute('''
                INSERT INTO artists (track_id, artist_name, artist_followers)
                VALUES (?, ?, ?)
            ''', (
                track_id,
                artist['name'],  
                artist['followers'] if 'followers' in artist else None  
            ))

        
        cursor.execute('''
            INSERT INTO audio_features (track_id, danceability, energy, key_int, loudness, mode_int, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms, time_signature)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            track_id,
            track_data['audio_features']['danceability'],
            track_data['audio_features']['energy'],
            track_data['audio_features']['key'],
            track_data['audio_features']['loudness'],
            track_data['audio_features']['mode'],
            track_data['audio_features']['speechiness'],
            track_data['audio_features']['acousticness'],
            track_data['audio_features']['instrumentalness'],
            track_data['audio_features']['liveness'],
            track_data['audio_features']['valence'],
            track_data['audio_features']['tempo'],
            track_data['audio_features']['duration_ms'],
            track_data['audio_features']['time_signature']
        ))

        conn.commit()

    finally:
        conn.close()


def main():
    server = 'DESKTOP-S9AVS0N\SQLEXPRESS'  
    database = 'spotify_data_4'  
    driver = 'ODBC Driver 17 for SQL Server'  

    create_tables(server, database, driver)

    for track_data in track_data_list:
        insert_track_data(server, database, driver, track_data)
    
if __name__ == '__main__':
    main()


IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'UQ__tracks__24ECC82FC2945813'. Cannot insert duplicate key in object 'dbo.tracks'. The duplicate key value is (6Ep6BzIOB9tz3P4sWqiiAB). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")